In [1]:
import torch


from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_dense_adj
from torch.utils.data import DataLoader, random_split

from torch_geometric.nn.pool import global_mean_pool, global_max_pool, global_add_pool

In [2]:
from helpers import CVFConfigDataset

In [3]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        # x = torch.sparse.mm(adjacency, x)
        x = torch.matmul(adjacency, x)
        return x

In [4]:
# dataset = CVFConfigDataset(
#     "small_graph_test_config_rank_dataset.csv", "small_graph_edge_index.json", 4
# )
# dataset = CVFConfigDataset(
#     "graph_1_config_rank_dataset.csv", "graph_1_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_4_config_rank_dataset.csv", "graph_4_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_5_config_rank_dataset.csv", "graph_5_edge_index.json", 9
# )
dataset = CVFConfigDataset(
    "graph_6b_config_rank_dataset.csv", "graph_6b_edge_index.json", 5
)
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n9_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n9_edge_index.json",
#     8
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n12_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n12_edge_index.json",
#     10
# )
train_size = int(0.75 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
# data_loader = DataLoader(dataset, batch_size=128, shuffle=False)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, edge_index):
        h = self.gcn1(x, edge_index)
        h = torch.relu(h)
        h = self.gcn2(h, edge_index)
        h = torch.relu(h)
        h = self.out(h)
        h = global_mean_pool(h, torch.zeros(h.size(1)).to(device).long())
        return h

    def fit(self, data_loader, epochs):
        # criterion = torch.nn.CrossEntrop yLoss()
        # criterion = torch.nn.MSELoss()
        criterion_sum = torch.nn.MSELoss(reduction='sum')
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        edge_index = dataset.edge_index.t().to(device)
        for epoch in range(1, epochs + 1):
            total_loss = 0
            count = 0
            max_loss = -1
            for _, batch in enumerate(data_loader, 1):
                x = batch[0].to(device)
                y = batch[1].to(device)
                y = y.unsqueeze(0).reshape(-1, 1, 1).float()
                optimizer.zero_grad()
                out = self(x, edge_index)
                # print("output", out, "y", y)
                # loss = criterion(out, y)
                loss = criterion_sum(out, y)
                total_loss += loss
                avg_loss = loss/len(batch)
                if avg_loss > max_loss:
                    max_loss = avg_loss
                count += len(batch)
                loss.backward()
                optimizer.step()
                # if i % 10 == 0:
                #     print("Iter", i, "| Loss:", avg_loss / count)

            print("Epoch", epoch, "| Loss:", total_loss / count, "| Max Loss:", max_loss)


In [7]:
gnn = VanillaGNN(dataset.num_classes, 128, 1).to(device)
print(gnn)

gnn.fit(train_loader, epochs=25)

VanillaGNN(
  (gcn1): GCNConv(5, 128)
  (gcn2): GCNConv(128, 128)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
Epoch 1 | Loss: tensor(29.0039, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(168.3417, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 2 | Loss: tensor(10.5192, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(20.6254, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 3 | Loss: tensor(9.4408, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(18.5399, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 4 | Loss: tensor(7.9881, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(14.3746, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 5 | Loss: tensor(6.4805, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(11.2032, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 6 | Loss: tensor(5.1575, device='cuda:0', grad_fn=<DivBackward0>) | Max Loss: tensor(11.7226, device='cuda:0', grad_fn=<DivBackward0>)
Epoch 7 | Loss: tensor(

In [8]:
# testing
torch.no_grad()
torch.set_printoptions(profile="full")

total_matched = 0
edge_index = dataset.edge_index.t().to(device)

for batch in test_loader:
    x = batch[0].to(device)
    y = batch[1].to(device)
    y = y.unsqueeze(0).reshape(-1, 1, 1).float()
    predicted = gnn(x, edge_index)
    # predicted = predicted.argmax(dim=1)
    predicted = torch.round(predicted)
    matched = (predicted == y).sum().item()
    total_matched += matched
    

print("Total matched", total_matched, "out of", len(test_dataset), "| Accuracy", round(total_matched/len(test_dataset) * 100, 4), "%")

Total matched 2546 out of 2700 | Accuracy 94.2963 %
